In [1]:
%matplotlib inline
import glob, os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
pd.options.display.max_rows = 200

plt.rcParams['figure.figsize'] = [20, 10]
plt.rc('font', size=20)



# Loading Dataset

## IOT Devices

The dataset has been extracted at `ROOT_PATH`, containing the data for each IOT device. There are 9 devices in total. There are folders for each IOT device containing `benign_traffic.csv` and further two folders for `gafgyt_attacks` and `mirai_attacks`

In [4]:
ROOT_PATH = r'/mnt/data/khiz/dataset/00442'

IOT_DEVS = [ 'Danmini_Doorbell',
             'Ecobee_Thermostat',
#              'Ennio_Doorbell',
#              'B120N10_Baby_Mon',
#              '737E_Security_Cam',
#              '838_Security_Cam',
#              'Samsung_Webcam',
#              '1002_Security_Cam',
#              '1003_Security_Cam'
           ]

## Paths to CSVs
We create a python dictionary that contains all the paths to the CSVs for the respective IOT devices

In [5]:
PATHS_DICT = {}

for i in IOT_DEVS:
    PATHS_DICT[i] = {}
    print('[' + i + ']')
    iot_dir = os.path.join(ROOT_PATH, i)
    PATHS_DICT[i]['benign'] = os.path.join(iot_dir, 'benign_traffic.csv')
    print('  ', 'benign_traffic.csv')
    attacks = [ d for d in os.listdir(iot_dir)
                   if os.path.isdir( os.path.join(iot_dir, d)) ]
    for attack in attacks:
        attack_name = attack.split('_')[0]
        
        PATHS_DICT[i][attack_name] = {}
        attack_dir = os.path.join(iot_dir, attack)
        types = [ f for f in os.listdir(attack_dir)
                   if os.path.isfile( os.path.join(attack_dir, f)) ]
        print('  ', attack_name, ': ', types)
        for t in types:
            type_name = t.split('.')[0]
            PATHS_DICT[i][attack_name][type_name] = os.path.join(attack_dir,t)

[Danmini_Doorbell]
   benign_traffic.csv
   gafgyt :  ['combo.csv', 'junk.csv', 'scan.csv', 'tcp.csv', 'udp.csv']
   mirai :  ['ack.csv', 'scan.csv', 'syn.csv', 'udp.csv', 'udpplain.csv']
[Ecobee_Thermostat]
   benign_traffic.csv
   gafgyt :  ['combo.csv', 'junk.csv', 'scan.csv', 'tcp.csv', 'udp.csv']
   mirai :  ['ack.csv', 'scan.csv', 'syn.csv', 'udp.csv', 'udpplain.csv']


## Loading CSVs into Pandas
We will create pandas dataframe for each botnet attack on each IOT device. For each datafram we will combine the attack traffic with the benign traffic add the following two columns:
- `traffic_type` : benign or attack (0 or 1)
- `attack_type`  : type of attack e.g, ack, scan etc.

In [6]:
nbaiot_dict = {}
for iot_dev in PATHS_DICT:
    nbaiot_dict[iot_dev] = {}
    b_df = pd.read_csv(PATHS_DICT[iot_dev]['benign'])
    b_df['traffic_type'] = 'benign'
    b_df['attack_type'] = 'benign'
    #nbaiot_dict[iot_dev] = nbaiot_dict[iot_dev].append(df)
    for botnet in [ b for b in PATHS_DICT[iot_dev] if b != 'benign']:
        nbaiot_dict[iot_dev][botnet] = pd.DataFrame()
        nbaiot_dict[iot_dev][botnet] = nbaiot_dict[iot_dev][botnet].append(b_df)
        for attack in PATHS_DICT[iot_dev][botnet]:
            a_df = pd.read_csv(PATHS_DICT[iot_dev][botnet][attack])
            a_df['traffic_type'] = 'attack'
            a_df['attack_type'] = attack
            nbaiot_dict[iot_dev][botnet] = nbaiot_dict[iot_dev][botnet].append(a_df)

In [7]:
for k in nbaiot_dict:
    print(k)
    for b in nbaiot_dict[k]:
        print('  ',b,':','<benign+attack>')

Danmini_Doorbell
   gafgyt : <benign+attack>
   mirai : <benign+attack>
Ecobee_Thermostat
   gafgyt : <benign+attack>
   mirai : <benign+attack>


# Modeling

### Pre processing
Convert the target column `traffic_type` to integer 0 = benigh, 1 = attack 

In [8]:
for dev in nbaiot_dict:
    for botnet in nbaiot_dict[dev]:
        nbaiot = nbaiot_dict[dev][botnet]

        nbaiot.loc[ nbaiot['traffic_type']=='attack','traffic_type' ]=1
        nbaiot.loc[ nbaiot['traffic_type']=='benign','traffic_type' ]=0
        nbaiot['traffic_type'] = nbaiot['traffic_type'].astype(int)
        
        nbaiot_dict[dev][botnet] = nbaiot

### Classifiers and training functions

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


In [14]:
classfiers = {
    'KNN' : KNeighborsClassifier(),
    'RFR' : RandomForestClassifier(),
    'DTR' : DecisionTreeClassifier(),
    'ADB' : AdaBoostClassifier(),
    
}


def train_test_report( X_train, X_test, y_train, y_test, avg='macro avg' ):

    result = []
    for clf_name in classfiers:
        clf = classfiers[clf_name]
        clf.fit(X_train, y_train)
        
        y_preds = clf.predict(X_test)
        
        clrp = classification_report(y_test, y_preds, output_dict=True)
        conf_mat = confusion_matrix(y_test, y_preds)
        fp = conf_mat[0][1]
        fn = conf_mat[1][0]
        
        result.append([
            clf_name,
            clrp['accuracy'],
            clrp[avg]['precision'],
            clrp[avg]['recall'],
            clrp[avg]['f1-score'],
            fp,
            fn
        ])

    return result

## Main Loop

In [15]:
all_results = [
    ['TRAIN DEVICE', 'TEST DEVICE', 'BOTNET', 'CLASSIFIER', 'ACCURACY', 'PRECISION', 'RECALL', 'F1-SCORE', 'FALSE-P', 'FALSE-N' ]
]

for train_dev in nbaiot_dict:
    print('TRAIN_DEV=',train_dev)
    for test_dev in nbaiot_dict:
        print('  TEST_DEV=',test_dev)
        for botnet in set(nbaiot_dict[test_dev].keys()).intersection(nbaiot_dict[train_dev].keys()):
            
            print("================", '/'.join([train_dev,test_dev,botnet]), "================")
            
            train_df = nbaiot_dict[train_dev][botnet]
            test_df  = nbaiot_dict[test_dev][botnet]
            
            X_train = train_df.drop(["attack_type", "traffic_type" ], axis=1)
            y_train = train_df['traffic_type']
            X_test  = test_df.drop(["attack_type", "traffic_type" ], axis=1)
            y_test  = test_df['traffic_type']
            
            rep = train_test_report( X_train, X_test, y_train, y_test )
            
            for r in rep:
                print(r)
                all_results.append( [train_dev, test_dev, botnet] + r )
            
# for dev in nbaiot_dict:
#     for botnet in nbaiot_dict[dev]:
#         nbaiot = nbaiot_dict[dev][botnet]
        
#         print("================", dev, "/", botnet, "================")

#         X = nbaiot.drop( ["attack_type", "traffic_type" ], axis=1 )
#         y = nbaiot["traffic_type"]

#         rep = train_test_report( X, y)
#         for r in rep:
#             print(r)
#             all_results.append( [dev, botnet] + r )

#         print("================================================================")
#         print("")
        

TRAIN_DEV= Danmini_Doorbell
  TEST_DEV= Danmini_Doorbell
================ Danmini_Doorbell/Danmini_Doorbell/mirai ================
['KNN', 0.9992232572457984, 0.9964852265685675, 0.9976053314324629, 0.9970445351315236, 212, 333]
['RFR', 1.0, 1.0, 1.0, 1.0, 0, 0]
['DTR', 1.0, 1.0, 1.0, 1.0, 0, 0]
['ADB', 1.0, 1.0, 1.0, 1.0, 0, 0]
================ Danmini_Doorbell/Danmini_Doorbell/gafgyt ================
['KNN', 0.999019656033075, 0.9973456990042966, 0.9984712501152531, 0.9979075575112704, 113, 246]
['RFR', 1.0, 1.0, 1.0, 1.0, 0, 0]
['DTR', 1.0, 1.0, 1.0, 1.0, 0, 0]
['ADB', 1.0, 1.0, 1.0, 1.0, 0, 0]
  TEST_DEV= Ecobee_Thermostat
================ Danmini_Doorbell/Ecobee_Thermostat/mirai ================
['KNN', 0.9952517487044166, 0.9263135696352905, 0.986901947611508, 0.9544755594532957, 287, 2207]
['RFR', 0.9999828651717481, 0.9999540391702109, 0.9996939828919046, 0.9998239753802531, 8, 1]
['DTR', 0.9999504993850501, 0.9991214142163916, 0.9998631545958616, 0.999491994165026, 3, 23]
['AD

In [ ]:
res_df =pd.DataFrame(all_results, index=None)

res_df, res_df.columns = res_df[1:] , res_df.iloc[0]

res_df

In [ ]:
# Write to csv
res_csv = res_df.to_csv(index=False)

print(res_csv, file=open('02_results.csv', 'w'))


### Load results from CSV

In [ ]:
res_df = pd.read_csv('04_results.csv')

res_df = res_df[['DEVICE', 'BOTNET', 'CLASSIFIER', 'ACCURACY', 'PRECISION', 'FALSE-P', 'FALSE-N' ]]

res_df

In [ ]:
res_df.set_index(['DEVICE', 'BOTNET', 'CLASSIFIER'], inplace=True)
res_df

In [ ]:
res_df0 = res_df.unstack(level=1)
res_df0

In [ ]:
res_df1 = res_df0.unstack(level=1)
res_df1

In [ ]:
res_df1.plot(kind='bar')